# Image segmentation with u-net

Creating image segmentation model with keras and u-net model. Model is built then converted for tesnorflow JS, for use in the browser.

Using [matting-human-datasets](https://www.kaggle.com/laurentmih/aisegmentcom-matting-human-datasets); using a 15,000 image sample resized to (400, 300).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split

from skimage.transform import resize

from keras.callbacks import EarlyStopping, ModelCheckpoint

from batches import batch_generator, test_generator, expand_path, load_images, load_image_masks
from utils import show_images
from model import build_u_net_model

%matplotlib inline

Using TensorFlow backend.


In [2]:
df_all = pd.read_csv('./dataset/images.csv')
print(df_all.size)

14999


## Original images

For each image we have the original and a copy with the background removed. We will used the subtracted image to generate background masks.

In [4]:
taster_ids = list(df_all['image_id'][:12])
size = len(taster_ids)

images = load_images(taster_ids, (size, 400, 300, 3))
alphas = load_images(taster_ids, (size, 400, 300, 4), kind='matting')
masks =  load_image_masks(taster_ids, (size, 400, 300, 3))

show_images([images, alphas, masks], titles=['img', 'matting', 'mask'])

interactive(children=(IntSlider(value=5, description='x', max=11), Output()), _dom_classes=('widget-interact',…

## Splitting into train, validation and test sets

* test: 500
* training: 80%
* validation: 20%

Also resize images to 128x128.

In [5]:
train_path = './dataset/train.csv.gz'
validation_path = './dataset/validation.csv.gz'
test_path = './dataset/test.csv.gz'

In [9]:
df_all = df_all.sample(frac=1, random_state=1)
df_data, df_test = df_all[500:], df_all[:500]

df_train, df_validation = train_test_split(df_data, test_size=0.2, random_state=101)

df_train = df_train.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

print('training data:', df_train.shape)
print('validation data:', df_validation.shape)
print('testing data:', df_test.shape)

df_train.to_csv(train_path, compression='gzip', index=False)
df_validation.to_csv(validation_path, compression='gzip', index=False)
df_test.to_csv(test_path, compression='gzip', index=False)

training data: (11599, 1)
validation data: (2900, 1)
testing data: (500, 1)


In [10]:
train_generator = batch_generator(train_path)
validation_generator = batch_generator(validation_path)

In [11]:
train_X, train_Y = next(train_generator)
print(train_X.shape, train_Y.shape)
show_images([train_X, train_Y], titles=['X (train img)', 'Y (train mask)'])

(10, 128, 128, 3) (10, 128, 128, 1)


interactive(children=(IntSlider(value=4, description='x', max=9), Output()), _dom_classes=('widget-interact',)…

In [12]:
val_X, val_Y = next(validation_generator)
print(val_X.shape, val_Y.shape)
show_images([val_X, val_Y], titles=['X (validation img)', 'Y (validation mask)'])

(10, 128, 128, 3) (10, 128, 128, 1)


interactive(children=(IntSlider(value=4, description='x', max=9), Output()), _dom_classes=('widget-interact',)…

## Training

In [8]:
model = build_u_net_model()

In [10]:
BATCH_SIZE = 100

num_train_samples = len(df_train)
num_val_samples = len(df_validation)
train_batch_size = BATCH_SIZE
val_batch_size = BATCH_SIZE

# determine numtrain steps
train_steps = np.ceil(num_train_samples / train_batch_size)
# determine num val steps
val_steps = np.ceil(num_val_samples / val_batch_size)
print(train_steps, val_steps)

# Initialize the generators
train_generator = batch_generator(train_path, batch_size=BATCH_SIZE)
validation_generator = batch_generator(validation_path, batch_size=BATCH_SIZE)


filepath = "model.h5"

earlystopper = EarlyStopping(patience=3, verbose=1)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True)

callbacks_list = [earlystopper, checkpoint]

history = model.fit_generator(train_generator, 
                              steps_per_epoch=train_steps, 
                              epochs=20, 
                              validation_data=validation_generator, 
                              validation_steps=val_steps,
                              verbose=1,
                              callbacks=callbacks_list)

116.0 29.0
Epoch 1/20
116/116 [==============================] - 860s 7s/step - loss: 0.2899 - val_loss: 0.3531

Epoch 00001: val_loss improved from inf to 0.35305, saving model to model.h5
Epoch 2/20
116/116 [==============================] - 834s 7s/step - loss: 0.1559 - val_loss: 0.2812

Epoch 00002: val_loss improved from 0.35305 to 0.28124, saving model to model.h5
Epoch 3/20
116/116 [==============================] - 871s 8s/step - loss: 0.1150 - val_loss: 0.4668

Epoch 00003: val_loss did not improve from 0.28124
Epoch 4/20
116/116 [==============================] - 818s 7s/step - loss: 0.0920 - val_loss: 0.1206

Epoch 00004: val_loss improved from 0.28124 to 0.12058, saving model to model.h5
Epoch 5/20
116/116 [==============================] - 841s 7s/step - loss: 0.0782 - val_loss: 0.1059

Epoch 00005: val_loss improved from 0.12058 to 0.10590, saving model to model.h5
Epoch 6/20
116/116 [==============================] - 823s 7s/step - loss: 0.0670 - val_loss: 0.0848

Epoch 

## Predictions

In [13]:
test_gen = test_generator(test_path, batch_size=1)

model.load_weights('model.h5')
predictions = model.predict_generator(test_gen, 
                                      steps=len(df_test),  
                                      verbose=1)

500/500 [==============================] - 23s 47ms/step


In [14]:
predictions.shape

(500, 128, 128, 1)

In [15]:
# get the list of images
test_id_list = list(df_test['image_id'])


for i, image_id in enumerate(test_id_list):
    if i == 0:
        # get a predicted mask
        image = predictions[i]
        # resize
        preds = resize(image, (400, 300))
        # reshape
        preds = preds.reshape((1, 400, 300, 1))
    else:
        # get a predicted mask
        image = predictions[i]
        # resize
        image = resize(image, (400, 300))        
        # reshape
        image = image.reshape((1, 400, 300, 1))
        # stack the images
        preds = np.vstack((preds, image))


preds.shape

(500, 400, 300, 1)

In [20]:
# Threshold the predictions

preds_test_thresh = (preds >= 0.5).astype(np.uint8)

preds_test_thresh.shape

print(preds_test_thresh.min())
print(preds_test_thresh.max())

0
1


In [21]:
# simply multiply by 255
alpha_preds = preds_test_thresh * 255

print(alpha_preds.min())
print(alpha_preds.max())

0
255


In [22]:
x_test_dims = (len(df_test), 400, 300, 3)
X_test_orig = load_images(test_id_list, x_test_dims)

In [23]:
print(X_test_orig.shape)
print(alpha_preds.shape)

(500, 400, 300, 3)
(500, 400, 300, 1)


In [24]:
predicted_masks = np.concatenate((X_test_orig, alpha_preds), axis=-1)

predicted_masks.shape

(500, 400, 300, 4)

## Displaying predicted masks

In [25]:
show_images([X_test_orig, predicted_masks, alpha_preds], titles=["orignal", "predicted mask", "generated output"])

interactive(children=(IntSlider(value=249, description='x', max=499), Output()), _dom_classes=('widget-interac…

In [41]:
!tensorflowjs_converter --input_format keras model.h5 tfjs/model

## further questions

* How to smooth the mask results, can this be done in the front end easily or would it be better to retrain model with smoothed masks? 
* Gradients in masks, how would a feathering effect work?
* How to extract proraits from landscape images?
* How to autocrop images to focus on people?
* How will this work with multiple people in the frame?